In [ ]:
#!pip install draversal

In [ ]:
!wget https://raw.githubusercontent.com/markomanninen/draversal/main/draversal.py

In [ ]:
from draversal import DictTraversal, DictSearchQuery
import draversal
import inspect

In [ ]:
method_docstrings = {}

# For module-level functions
module = inspect.getmodule(draversal)  # Replace DictTraversal with the actual class name
for func_name in sorted(dir(module)):
    if func_name in ["contextmanager"]:
        continue
    func = getattr(module, func_name)
    if inspect.isfunction(func):
        method_docstrings[func_name] = func.__doc__
    elif inspect.isclass(func):
        method_docstrings[func_name] = func.__doc__

private_methods = ['__init__', '__iter__', '__next__', '__pos__', '__neg__', '__getitem__', '__delitem__']

for method_name in sorted(dir(DictTraversal)):
    method = getattr(DictTraversal, method_name)
    if callable(method) and method.__qualname__.split('.')[0] == 'DictTraversal' and method_name not in ['BackwardIterator', 'fromkeys']:
        if not method_name.startswith('_') or method_name in private_methods:
            method_docstrings["DictTraversal." + method_name] = method.__doc__

for method_name in sorted(dir(DictSearchQuery)):
    method = getattr(DictSearchQuery, method_name)
    if callable(method) and method.__qualname__.split('.')[0] == 'DictSearchQuery' and method_name not in ['OPERATOR_MAP', 'fromkeys']:
        if not method_name.startswith('_') or method_name in private_methods:
            method_docstrings["DictSearchQuery." + method_name] = method.__doc__

In [ ]:
method_docstrings.keys()

In [ ]:
from IPython.core.display import display, Markdown
import re

def params_to_string(parsed_params):
    param_lines = []
    for name, details in parsed_params.items():
        param_type = details['type']
        description = details['description']
        if param_type:
            param_line = f"- __{name}__ ({param_type}): {description}"
        else:
            param_line = f"- __{name}__: {description}"
        param_lines.append(param_line)
    return '\n'.join(param_lines) + '\n'

def parse_parameters(param_text):
    parsed_params = {}
    lines = param_text.strip().split('\n')
    for line in lines:
        # Split by the first colon to separate name and type from description
        name_type, description = line.split(':', 1)
        # Further split name and type if available
        name_type_parts = name_type.split('(', 1)
        name = name_type_parts[0].strip()
        param_type = name_type_parts[1].rstrip(')') if len(name_type_parts) > 1 else None
        # Store in dictionary
        parsed_params[name] = {'type': param_type, 'description': description.strip()}
    return parsed_params

# Function to format a docstring into a complete Markdown template
def complete_format_docstring_to_markdown(method_name, docstring):
    if not docstring:
        return
    lines = docstring.strip().split('\n')
    description, params, returns, raises, behavior, examples, note, yields, attributes = [], [], '', [], [], [], [], '', []
    section = None
    space = "           "
    if method_name in ["validate_data", "prev", "root", "first", "last", "demo", "DictTraversal", "DictSearchQuery", "flatten_dict", "reconstruct_item"]:
        space = "       "
    for line in lines:
        codeline = line.rstrip().replace(space, "", 1)
        line = line.strip()
        if line.startswith('Parameters:'):
            section = 'params'
        elif line.startswith('Attributes:'):
            section = 'attributes'
        elif line.startswith('Returns:'):
            section = 'returns'
        elif line.startswith('Raises:'):
            section = 'raises'
        elif line.startswith('Behavior:'):
            section = 'behavior'
        elif line.startswith('Example:'):
            section = 'examples'
        elif line.startswith('Note:'):
            section = 'note'
        elif line.startswith('Yields:'):
            section = 'yield'
        else:
            if section == 'params':
                params.append(line)
            elif section == 'attributes':
                attributes.append(line)
            elif section == 'returns':
                returns = line
            elif section == 'raises':
                raises.append(line)
            elif section == 'behavior':
                behavior.append(codeline)
            elif section == 'examples':
                examples.append(codeline)
            elif section == 'note':
                note.append(line)
            elif section == 'yield':
                yields = line
            else:
                description.append(line)

    markdown_lines = ['\n\n']
    method_or_class = 'Method' if method_name not in ['DictTraversal', 'DictSearchQuery'] else 'Class'
    markdown_lines.append(f'# {method_or_class}: `{method_name}`\n\n')
    if description:
        markdown_lines.extend(['## Description\n', '\n'.join(description) + '\n', ''])
    if params:
        markdown_lines.extend(['## Parameters\n', params_to_string(parse_parameters('\n'.join(params))), ''])
    if attributes:
        #print(method_name, attributes)
        markdown_lines.extend(['## Attributes\n', params_to_string(parse_parameters('\n'.join(attributes))), ''])
    if returns:
        markdown_lines.extend(['## Returns\n', f'{returns}\n', ''])
    if yields:
        markdown_lines.extend(['## Yields\n', f'{yields}\n', ''])
    if raises:
        markdown_lines.extend(['## Raises\n', params_to_string(parse_parameters('\n'.join(raises))), ''])
    if behavior:
        markdown_lines.extend(['## Behavior\n', '\n'.join(behavior) + '\n', ''])
    if note:
        markdown_lines.extend(['## Note\n', '\n'.join(note) + '\n', ''])
    if examples:
        markdown_lines.extend(['## Example\n', '\n'.join(examples) + '\n', ''])
    return '\n---\n\n' + ''.join(markdown_lines)

all_formatted_md = ""
# Loop through each method and display its formatted docstring
for method_name, docstring in method_docstrings.items():
    #print(f"method_name: {method_name}")
    formatted_md = complete_format_docstring_to_markdown(method_name, docstring)
    if formatted_md:
        display(Markdown(formatted_md))
        all_formatted_md += formatted_md

with open("draversal2_docs.md", "w") as f:
    f.write(all_formatted_md)